# Optimization with Constraint Learning: A Chemotherapy case study

## The Chemotherapy Problem  

<font size="4"> Problem Description</font>   

In this case study, we extend the work of  [Bertsimas et al. (2016)](https://pubsonline.informs.org/doi/10.1287/mnsc.2015.2363) in the design of chemotherapy regimens for advanced gastric cancer. Given a new study cohort and study characteristics, [Bertsimas et al. (2016)](https://pubsonline.informs.org/doi/10.1287/mnsc.2015.2363) optimize a chemotherapy regimen to maximize the cohort’s survival subject to a constraint on overall toxicity. Chemotherapy regimens are particularly challenging to optimize, since they involve multiple drugs given at potentially varying dosages, and they present risks for multiple adverse events that must be managed. This example highlights the generalizability of OptiCL to complex domains with multiple decisions and learned functions. 

<div>
<img src="figures/lab.jpg" width="400"/>
</div>

We have several learned constraints which must be simultaneously satisfied, and we also learn the objective function directly as a predictive model. The use of clinical trial data forces us to consider each cohort as an observation, rather than an individual, since only aggregate measures are available. Thus, our model optimizes a cohort’s treatment.

In [1]:
import pandas as pd
import numpy as np
import time
import sys
import os
import opticl
import utils_gastric as gi

# Optimization modelling
from pyomo import environ
from pyomo.environ import *

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

## Optimization model

The contextual variables ($\boldsymbol{w}$) consist of various cohort and study summary variables. The inclusion of fixed, i.e., non-optimization, features allows us to account for differences in baseline health status and risk across study cohorts. These features are included in the predictive models but then are fixed in the optimization model to reflect the group for whom we are generating a prescription. We assume that there are no unobserved confounding variables in this prescriptive setting.  

The treatment variables ($\boldsymbol{x}$) encode a chemotherapy regimen. A regimen is defined by a set of drugs, each with an administration schedule of potentially varied dosages throughout a chemotherapy cycle. We characterize a regimen by drug indicators and each drug's average daily dose and maximum instantaneous dose in the cycle.
\begin{align*}
    & \boldsymbol{x}_b^d = \mathbb{I}(\text{drug $d$ is administered}), \\ 
    & \boldsymbol{x}_a^d = \text{average daily dose of drug $d$}, \\
    & \boldsymbol{x}_i^d = \text{maximum instantaneous dose of drug $d$}.
\end{align*}

The outcomes of interest ($\boldsymbol{y}$) consist of overall survival, to be included as the objective ($y_{OS}$),and various toxicities, to be included as constraints   ($y_i, \ i \in \mathcal{Y}_C$).

\begin{align*}
    \min_{\boldsymbol{x},\boldsymbol{y}} \ & y_{OS} & \\
    \mbox{s.t.} & y_i \leq \tau_i, &  {i \in \mathcal{Y}_C}, \\ 
    & y_i = \hat{h}_i(\boldsymbol{x},\boldsymbol{w}), &  i \in \mathcal{Y}_C, \\
    & y_{OS} = \hat{h}_{OS}(\boldsymbol{x},\boldsymbol{w}), \\
    & \sum_{d} \boldsymbol{x}_b^d \leq 3, \\ 
    &  \boldsymbol{x}_b \in \{0,1\}^d, \\
     & \boldsymbol{x} \in \mathcal{X}(\boldsymbol{w}).
\end{align*}

In this case study, we learn the full objective. However, this model could easily incorporate deterministic components to optimize as additional weighted terms in the objective. We include one domain-driven constraint, enforcing a maximum regimen combination of three drugs. 

**Trust region**

The trust region, $\mathcal{X}(\boldsymbol{w})$, plays two crucial roles in the formulation. First, it ensures that the predictive models are applied within their valid bounds and not inappropriately extrapolated. It also naturally enforces a notion of "clinically reasonable" treatments. It prevents drugs from being prescribed at doses outside of previously observed bounds, and it requires that the drug combination must have been previously seen (although potentially in different doses). It is nontrivial to explicitly characterize what constitutes a realistic treatment, and the convex hull provides a data-driven solution that integrates directly into the model framework. Furthermore, the convex hull implicitly enforces logical constraints between the different dimensions of $\boldsymbol{x}$. For example, a drug's average and instantaneous dose must be 0, if the drug's binary indicator is set to 0: this does not need to be explicitly included as a constraint, since this is true of all observed treatment regimens. The only explicit constraint required here is that the indicator variables $\boldsymbol{x}_b$ are binary.

In [2]:
def init_conceptual_model(pt, N):
    model = ConcreteModel('chemo')

#     N = list(pt.keys())
    ########### STEP 1: Define Decision Variables ###########
    # Create x variable, and initialize empty y for outcome values
    model.x = Var(N,domain=NonNegativeReals)

    # Restrict some x to be binary
    x_binary = [i for i in N if '_Ind' in i]
    for i in x_binary:
        model.x[i].domain = Binary

    ########### STEP 2: Define Objective function ###########
        def obj_function(model):
            return 0
    model.OBJ = Objective(rule=obj_function, sense=minimize)

    ###### STEP 3: Add (optionally) any known constraints #####
    def constraint_rule1(model):
        return sum(model.x[i] for i in x_binary) <= 3
    model.Constraint1 = Constraint(rule=constraint_rule1)
    
    ###### STEP 4: Fix (optionally) any non-optimization variables #####
#     def constraint_rule2(model, i):
#         return model.x[i] == pt[i]
#     model.Constraint2 = Constraint(contex_vars, rule=constraint_rule2)

    return model

## Data loading

Load ```tox_summary```, which reports quantiles for various toxicities as candidate upper bounds

In [3]:
tox_summary = pd.read_csv('processed-data/gastric_toxicity_summary.csv')
tox_summary.set_index('outcome', inplace = True)

Load training and testing datasets for each outcome. Note that in this case, the training data (X) is the same for all outcomes, but our framework is generic to independent training sets. Thus, we load each as a separate file for this example.

We include several "dose-limiting toxicities" (DLTs) for our constraint set: Grade 3/4 constitutional toxicity, gastrointestinal toxicity, and infection, as well as Grade 4 blood toxicity. As the name suggests, these are chemotherapy side effects that are severe enough to affect the course of treatment. We also consider incidence of any dose-limiting toxicity ("Any DLT"), which aggregates over a superset of these DLTs.

In [4]:
outcomes = gi.outcomes
datasets_train = {}
datasets_test = {}

# Training datasets
for o in outcomes:
    data = pd.read_csv(f'processed-data/data_train_{o}.csv')
    y = data[o]
    X = data.drop([o], inplace=False, axis=1)
    datasets_train[o] = (X, y)
    
# Testing datasets
for o in outcomes:
    data = pd.read_csv(f'processed-data/data_test_{o}.csv')
    y = data[o]
    X = data.drop([o], inplace=False, axis=1)
    datasets_test[o] = (X, y)

In [5]:
outcomes

['Neutro4',
 'OTHER_34',
 'GINONV_34',
 'CONSTITUTIONAL_34',
 'INFECTION_34',
 'DLT_PROP',
 'OS']

## Specify the outcomes and relevant parameters for ML training + optimization
```outcome_list``` is a dictionary where each key is an outcome, and each value is a dictionary corresponding to that outcome. The outcome-specific dictionary specifies relevant parameters and data for (1) ML model training and (2) the final optimization problem.

In [6]:
## ML training parameters: 
# specify models to consider, and optionally specify a grid to search over in CV (if None, will use default grid)
alg_dict = {'cart': None, 'linear':None, 'rf': None}
# specify whether we train a single model (bs = 0) or bootstrapped models (bs >= 1) for the outcome
bs = 0
# if training multiple models, select the single best (gr = False) or group together as an ensemble (gr = True)
gr=False
# if grouping an ensemble, specify proportion that can violate (or "average" to constrain mean)
viol_rule = 0.5

Add all constraint outcomes to dictionary. For each toxicity, we want to enforce that the toxicity is below a certain quantile of the toxicities observed in the data (which we loaded in tox_summary). Here, we select the 70th percentile, givenin column ```quantile_0.7```.

We also specify the training and testing data for ML model training as well as reporting test set metrics. dataset_path specifies the data that will be used to define the trust region in the downstream optimization task.

In [7]:
constraints_embed = gi.outcomes[:-1]
ub_quantile = 'quantile_0.7'

outcome_list = {outcome: {'lb':None, 'ub':tox_summary.loc[outcome,ub_quantile],
                          'objective_weight':0,'group_models':gr,
                        'task_type': 'continuous', 'alg_list':alg_dict, 'bootstrap_iterations':bs,
                        'X_train':datasets_train[outcome][0], 'y_train':datasets_train[outcome][1], 
                          'X_test':datasets_test[outcome][0], 'y_test':datasets_test[outcome][1],
                        'dataset_path': f'processed-data/data_train_{outcome}.csv'} 
                for outcome in constraints_embed}

Add the objective outcome to dictionary. In this case, the only outcome that we are seeking to optimize is overall survival (OS). We want to maximize survival; since the optimization formulation assumes that the objective will be minimized, we set the weight to -1.

We similarly specify the training and testing sets and the trust region dataset path.

In [8]:
outcome_list['OS'] = {'lb':None, 'ub':None, 
                      'objective_weight':-1,'group_models':gr,
                        'task_type': 'continuous', 'alg_list':alg_dict, 'bootstrap_iterations':bs,
                       'X_train':datasets_train['OS'][0], 'y_train':datasets_train['OS'][1], 
                      'X_test':datasets_test['OS'][0], 'y_test':datasets_test['OS'][1],
                       'dataset_path':'processed-data/data_train_OS.csv'}

In [9]:
print("Algorithms = %s" % alg_dict)
print("Bootstrap iterations = %d" % bs)
print("Violation rule = %s" % str(viol_rule))
code_version = 'AAAI-23_CHEMOexample'

version = 'vAAAI-23_CHEMOexample'

Algorithms = {'cart': None, 'linear': None, 'rf': None}
Bootstrap iterations = 0
Violation rule = 0.5


## Train candidate ML models and select models to embed 

In [10]:
performance = opticl.train_ml_models(outcome_list, version)
if not os.path.exists('results'):
    os.makedirs('results')
performance.to_csv('results/%s_performance.csv' % (code_version))
# performance = pd.read_csv('results/%s_performance.csv' % (code_version))

print("\nPreparing model master")
if viol_rule == 'average':
    gr_method = 'average'
    max_viol = None
    print("Group method = %s" % (gr_method))
    gr_string = 'average'
else: 
    gr_method = 'violation'
    max_viol = float(viol_rule)
    print("Group method = %s (violation limit = %.2f)" % (gr_method, max_viol))
    gr_string = 'violation_%.2f' % max_viol

Learning a model for Neutro4
No bootstrap - training on full training data
training Neutro4 with cart
------------- Initialize grid  ----------------
------------- Running model  ----------------
Algorithm = cart, metric = None
saving... results/cart_Neutro4_trained.pkl
------------- Model evaluation  ----------------
-------------------training evaluation-----------------------
Train MSE: 0.0200432326421685
Train R2: 0.3021467451768366
-------------------testing evaluation-----------------------
Test MSE: 0.019041832379617617
Test R2: -0.27527754538206173

training Neutro4 with linear
------------- Initialize grid  ----------------
------------- Running model  ----------------
Algorithm = linear, metric = None
saving... results/linear_Neutro4_trained.pkl
------------- Model evaluation  ----------------
-------------------training evaluation-----------------------
Train MSE: 0.022053049635415448
Train R2: 0.23217014233157096
-------------------testing evaluation-----------------------


saving... results/linear_DLT_PROP_trained.pkl
------------- Model evaluation  ----------------
-------------------training evaluation-----------------------
Train MSE: 0.025096590054807834
Train R2: 0.14016226847086855
-------------------testing evaluation-----------------------
Test MSE: 0.02901677060355753
Test R2: 0.11790077351021544

training DLT_PROP with rf
------------- Initialize grid  ----------------
------------- Running model  ----------------
Algorithm = rf_shallow, metric = None
saving... results/rf_shallow_DLT_PROP_trained.pkl
------------- Model evaluation  ----------------
-------------------training evaluation-----------------------
Train MSE: 0.013322680881295186
Train R2: 0.5435497937432003
-------------------testing evaluation-----------------------
Test MSE: 0.025273454578621846
Test R2: 0.23169621323078016

Learning a model for OS
No bootstrap - training on full training data
training OS with cart
------------- Initialize grid  ----------------
------------- Runn

In [11]:
performance

,save_path,seed,cv_folds,task,parameters,best_params,valid_score,train_score,train_r2,test_score,test_r2,auc_threshold,auc_train,auc_test,outcome,outcome_label,alg,bootstrap_iteration
0,results/cart/vAAAI-23_CHEMOexample_Neutro4_mod...,1,5,continuous,"{'max_depth': [3, 4, 5, 6, 7, 8, 9, 10], 'min_...","{'max_depth': 3, 'max_features': 0.6, 'min_sam...",-0.025203,0.020043,3.021467e-01,0.019042,-0.275278,0.150000,0.710780,0.682927,Neutro4,Neutro4,cart,0
0,results/linear/vAAAI-23_CHEMOexample_Neutro4_m...,1,5,continuous,"{'alpha': [0.1, 1, 10, 100, 1000], 'l1_ratio':...","{'alpha': 1, 'l1_ratio': 0.5000000000000001}",-0.026577,0.022053,2.321701e-01,0.010659,0.286132,0.150000,0.738783,0.834146,Neutro4,Neutro4,linear,0
0,results/rf/vAAAI-23_CHEMOexample_Neutro4_model...,1,5,continuous,"{'n_estimators': [10, 25], 'max_features': ['a...","{'max_depth': 2, 'max_features': 'auto', 'n_es...",-0.025467,0.014842,4.832348e-01,0.013331,0.107164,0.150000,0.774466,0.691057,Neutro4,Neutro4,rf,0
0,results/cart/vAAAI-23_CHEMOexample_OTHER_34_mo...,1,5,continuous,"{'max_depth': [3, 4, 5, 6, 7, 8, 9, 10], 'min_...","{'max_depth': 3, 'max_features': 0.6, 'min_sam...",-0.000824,0.000546,3.264830e-01,0.001372,-0.036475,0.100811,0.815842,0.602339,OTHER_34,OTHER_34,cart,0
0,results/linear/vAAAI-23_CHEMOexample_OTHER_34_...,1,5,continuous,"{'alpha': [0.1, 1, 10, 100, 1000], 'l1_ratio':...","{'alpha': 1, 'l1_ratio': 0.1}",-0.000822,0.000668,1.754051e-01,0.001187,0.103643,0.100811,0.725267,0.794591,OTHER_34,OTHER_34,linear,0
0,results/rf/vAAAI-23_CHEMOexample_OTHER_34_mode...,1,5,continuous,"{'n_estimators': [10, 25], 'max_features': ['a...","{'max_depth': 4, 'max_features': 'auto', 'n_es...",-0.000791,0.000293,6.379431e-01,0.001191,0.100149,0.100811,0.872159,0.669591,OTHER_34,OTHER_34,rf,0
0,results/cart/vAAAI-23_CHEMOexample_GINONV_34_m...,1,5,continuous,"{'max_depth': [3, 4, 5, 6, 7, 8, 9, 10], 'min_...","{'max_depth': 5, 'max_features': 0.4, 'min_sam...",-0.005311,0.003570,3.708147e-01,0.007473,-0.010506,0.106000,0.798872,0.608918,GINONV_34,GINONV_34,cart,0
0,results/linear/vAAAI-23_CHEMOexample_GINONV_34...,1,5,continuous,"{'alpha': [0.1, 1, 10, 100, 1000], 'l1_ratio':...","{'alpha': 0.1, 'l1_ratio': 0.7000000000000001}",-0.005307,0.003973,2.998640e-01,0.005832,0.211368,0.106000,0.754073,0.726608,GINONV_34,GINONV_34,linear,0
0,results/rf/vAAAI-23_CHEMOexample_GINONV_34_mod...,1,5,continuous,"{'n_estimators': [10, 25], 'max_features': ['a...","{'max_depth': 3, 'max_features': 'auto', 'n_es...",-0.005594,0.002719,5.208710e-01,0.005476,0.259445,0.106000,0.859492,0.697368,GINONV_34,GINONV_34,rf,0
0,results/cart/vAAAI-23_CHEMOexample_CONSTITUTIO...,1,5,continuous,"{'max_depth': [3, 4, 5, 6, 7, 8, 9, 10], 'min_...","{'max_depth': 3, 'max_features': 0.4, 'min_sam...",-0.005231,0.004121,3.285379e-01,0.004673,0.092400,0.081818,0.639873,0.605581,CONSTITUTIONAL_34,CONSTITUTIONAL_34,cart,0


Specify **global** trust region data and clustering model (if relevant). This will force the solution to lie within the convex hull of the datapoints given in 'data'. It will only enforce the convex hull condition on the features that appear in 'data': features not in the trust region dataset will not be constrained by the convex hull. 

In this case, we enforce the trust region *only* on the treatment features. We do not apply it to the contextual features.

In [12]:
tr_data = datasets_train['OS'][0][[i for i in gi.T_cols if i in X.columns]]
trust_region_specs = {'data': tr_data,
                      'clustering_model':None,
                      'enlargement':[0, 0, 0]}

Pull the test sample's data and define model master. If the outcome models are not grouped (i.e., not ensemble), we use the validation metrics from the cross-validation training procedure to select the best model for each outcome. Otherwise, all models will be used.

Select a test sample to optimize (i.e. a cohort of patients treated in 2008 or later), given as a row number of the test set.

In [13]:
patient_ID = 5 # 2
print("\nPatient %d" % patient_ID)


Patient 5


In [14]:
pt = datasets_test['OS'][0].loc[patient_ID, :]
features = pt.keys()

var_fts = [i for i in gi.T_cols if i in features]
context_fts = [i for i in gi.X_cols if i in features]

### Select fitted models  

In [15]:
mm = opticl.initialize_model_master(outcome_list)
mm.loc[outcomes,'group_method'] = gr_method
mm.loc[outcomes,'max_violation'] = max_viol
mm.loc[outcomes, 'trust_region'] = False
mm.loc[outcomes, 'var_features'] = [var_fts]
mm.loc[outcomes, 'contex_features'] = [{i:pt[i] for i in context_fts}]
model_master = opticl.model_selection(mm, performance)
model_master

,model,task,objective,lb,ub,features,var_features,contex_features,group_models,group_method,ensemble_weights,max_violation,trust_region,dataset_path,clustering_model,enlargement,SCM_counterfactuals
Neutro4,{'results/cart/vAAAI-23_CHEMOexample_Neutro4_m...,continuous,0,None,0.15,"Index(['Asia', 'N_Patient', 'FRAC_MALE', 'AGE_...","[Capecitabine_Ind, Carboplatin_Ind, Cisplatin_...","{'Asia': 1.0, 'N_Patient': 14.0, 'FRAC_MALE': ...",False,violation,None,0.5,False,processed-data/data_train_Neutro4.csv,None,[0],None
OTHER_34,{'results/rf/vAAAI-23_CHEMOexample_OTHER_34_mo...,continuous,0,None,0.100811,"Index(['Asia', 'N_Patient', 'FRAC_MALE', 'AGE_...","[Capecitabine_Ind, Carboplatin_Ind, Cisplatin_...","{'Asia': 1.0, 'N_Patient': 14.0, 'FRAC_MALE': ...",False,violation,None,0.5,False,processed-data/data_train_OTHER_34.csv,None,[0],None
GINONV_34,{'results/linear/vAAAI-23_CHEMOexample_GINONV_...,continuous,0,None,0.106,"Index(['Asia', 'N_Patient', 'FRAC_MALE', 'AGE_...","[Capecitabine_Ind, Carboplatin_Ind, Cisplatin_...","{'Asia': 1.0, 'N_Patient': 14.0, 'FRAC_MALE': ...",False,violation,None,0.5,False,processed-data/data_train_GINONV_34.csv,None,[0],None
CONSTITUTIONAL_34,{'results/cart/vAAAI-23_CHEMOexample_CONSTITUT...,continuous,0,None,0.081818,"Index(['Asia', 'N_Patient', 'FRAC_MALE', 'AGE_...","[Capecitabine_Ind, Carboplatin_Ind, Cisplatin_...","{'Asia': 1.0, 'N_Patient': 14.0, 'FRAC_MALE': ...",False,violation,None,0.5,False,processed-data/data_train_CONSTITUTIONAL_34.csv,None,[0],None
INFECTION_34,{'results/linear/vAAAI-23_CHEMOexample_INFECTI...,continuous,0,None,0.075,"Index(['Asia', 'N_Patient', 'FRAC_MALE', 'AGE_...","[Capecitabine_Ind, Carboplatin_Ind, Cisplatin_...","{'Asia': 1.0, 'N_Patient': 14.0, 'FRAC_MALE': ...",False,violation,None,0.5,False,processed-data/data_train_INFECTION_34.csv,None,[0],None
DLT_PROP,{'results/cart/vAAAI-23_CHEMOexample_DLT_PROP_...,continuous,0,None,0.637304,"Index(['Asia', 'N_Patient', 'FRAC_MALE', 'AGE_...","[Capecitabine_Ind, Carboplatin_Ind, Cisplatin_...","{'Asia': 1.0, 'N_Patient': 14.0, 'FRAC_MALE': ...",False,violation,None,0.5,False,processed-data/data_train_DLT_PROP.csv,None,[0],None
OS,{'results/rf/vAAAI-23_CHEMOexample_OS_model.cs...,continuous,-1,None,None,"Index(['Asia', 'N_Patient', 'FRAC_MALE', 'AGE_...","[Capecitabine_Ind, Carboplatin_Ind, Cisplatin_...","{'Asia': 1.0, 'N_Patient': 14.0, 'FRAC_MALE': ...",False,violation,None,0.5,False,processed-data/data_train_OS.csv,None,[0],None


## Solve the optimization problem

In [16]:
conceptual_model = init_conceptual_model(pt, var_fts)
final_model = opticl.optimization_MIP(conceptual_model, model_master, trust_region_specs)
opt = SolverFactory('glpk')
print('Solving...')
results = opt.solve(final_model) 
print('Done!')
print(results.solver.termination_condition)

Generating constraints for the trust region using 122 samples.
... Trust region defined.
Embedding constraints for Neutro4
Adding single model.
Embedding constraints for OTHER_34
Adding single model.
Embedding constraints for GINONV_34
Adding single model.
Embedding constraints for CONSTITUTIONAL_34
Adding single model.
Embedding constraints for INFECTION_34
Adding single model.
Embedding constraints for DLT_PROP
Adding single model.
Embedding objective function for OS
Adding single model.
Solving...
Done!
optimal


## Inspect the solution

What were the contextual features of this patient?

In [17]:
{i:pt[i] for i in context_fts}

{'Asia': 1.0,
 'N_Patient': 14.0,
 'FRAC_MALE': 0.5,
 'AGE_MED': 45.9,
 'Prior_Palliative_Chemo': 0.0,
 'Primary_Stomach': 1.0,
 'Primary_GEJ': 0.0,
 'ECOG_MEAN': 0.9045226007671588}

What drugs are recommended, and in what doses (average and instantaneous)?

In [18]:
for i in var_fts:
    val = value(final_model.x[i])
    if val > 1e-6:
        print("%s: %.3f" % (i, val))

Fluorouracil_Ind: 1.000
Leucovorin_Ind: 1.000
Paclitaxel_Ind: 1.000
Fluorouracil_Avg: 1500.000
Leucovorin_Avg: 375.000
Paclitaxel_Avg: 43.750
Fluorouracil_Inst: 2000.000
Leucovorin_Inst: 500.000
Paclitaxel_Inst: 175.000


What is the predicted value of each toxicity and objective?

In [19]:
for i in outcome_list:
    val = value(final_model.y[i])
    try: # print constraints
        print("%s: %.3f (limit = %.3f)" % (i, val, tox_summary.loc[i,ub_quantile]))
    except: # no bound for objective
        print("%s: %.3f" % (i, val))

Neutro4: 0.079 (limit = 0.150)
OTHER_34: 0.087 (limit = 0.101)
GINONV_34: 0.070 (limit = 0.106)
CONSTITUTIONAL_34: 0.060 (limit = 0.082)
INFECTION_34: 0.052 (limit = 0.075)
DLT_PROP: 0.527 (limit = 0.637)
OS: 11.306
